# Ingestion & Cleaning — USA Salaries

**Setup and Ingestion**

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder


pd.set_option('display.max_columns', 200)


ImportError: DLL load failed while importing lib: The specified procedure could not be found.

In [ ]:

df = pd.read_csv('data/ai_job_dataset0.csv')
df.head

<bound method NDFrame.head of         job_id                    job_title  salary_usd salary_currency  \
0      AI00001        AI Research Scientist       90376             USD   
1      AI00002         AI Software Engineer       61895             USD   
2      AI00003                AI Specialist      152626             USD   
3      AI00004                 NLP Engineer       80215             USD   
4      AI00005                AI Consultant       54624             EUR   
...        ...                          ...         ...             ...   
14995  AI14996            Robotics Engineer       38604             USD   
14996  AI14997  Machine Learning Researcher       57811             GBP   
14997  AI14998                 NLP Engineer      189490             USD   
14998  AI14999                   Head of AI       79461             EUR   
14999  AI15000     Computer Vision Engineer       56481             USD   

      experience_level employment_type company_location company_size 

In [ ]:
print("\nInitial DataFrame Info:")
df.info()



Initial DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   job_id                  15000 non-null  object 
 1   job_title               15000 non-null  object 
 2   salary_usd              15000 non-null  int64  
 3   salary_currency         15000 non-null  object 
 4   experience_level        15000 non-null  object 
 5   employment_type         15000 non-null  object 
 6   company_location        15000 non-null  object 
 7   company_size            15000 non-null  object 
 8   employee_residence      15000 non-null  object 
 9   remote_ratio            15000 non-null  int64  
 10  required_skills         15000 non-null  object 
 11  education_required      15000 non-null  object 
 12  years_experience        15000 non-null  int64  
 13  industry                15000 non-null  object 
 14  posting_date 

### Quick schema check — align to proposal columns
We expect the dataset to contain (per proposal):
- job_id, job_title, salary_usd, salary_currency, salary_local, experience_level,
- employment_type, job_category, company_location, company_size, employee_residence,
- remote_ratio, required_skills, education_required, years_experience, industry,
- posting_date, application_deadline, job_description_length, benefits_score

We'll display the actual columns present and map / rename where needed.

**Column Normalization**

In [ ]:
def canonicalize_columns(df_in):
    """Lowercases and standardizes column names."""
    df_out = df_in.copy()
    col_map = {c: c.strip().lower().replace('-', '_').replace(' ', '_') for c in df_out.columns}
    df_out = df_out.rename(columns=col_map)
    return df_out

df_cleaned = canonicalize_columns(df)
print('Canonical columns:\n', df_cleaned.columns.tolist())


Canonical columns:
 ['job_id', 'job_title', 'salary_usd', 'salary_currency', 'experience_level', 'employment_type', 'company_location', 'company_size', 'employee_residence', 'remote_ratio', 'required_skills', 'education_required', 'years_experience', 'industry', 'posting_date', 'application_deadline', 'job_description_length', 'benefits_score', 'company_name']


**Dropping Unnecessary Columns**

In [ ]:
cols_to_drop = ['salary_local', 'salary_currency']

existing_cols_to_drop = [col for col in cols_to_drop if col in df_cleaned.columns]
df_cleaned.drop(columns=existing_cols_to_drop, inplace=True)

print(f"Dropped columns: {existing_cols_to_drop}")
print(f"DataFrame shape after drops: {df_cleaned.shape}")


Dropped columns: ['salary_currency']
DataFrame shape after drops: (15000, 18)


In [ ]:
duplicate_count = df_cleaned.duplicated(subset=['job_id']).sum()

if duplicate_count > 0:
    print(f"Found {duplicate_count} duplicate job_id entries.")
else:
    print("No duplicates found for job_id.")



No duplicates found for job_id.


In [ ]:
missing_salary_count = df_cleaned['salary_usd'].isna().sum()
print(f"Rows with missing salary_usd: {missing_salary_count}")


Rows with missing salary_usd: 0


**Data Type Conversions**

In [ ]:

for dcol in ['posting_date', 'application_deadline']:
    if dcol in df_cleaned.columns:
        temp = pd.to_datetime(df_cleaned[dcol], errors='coerce')
        invalid_dates = temp.isna().sum()
        print(f"{dcol}: {invalid_dates} invalid date entries")

for ncol in ['years_experience', 'job_description_length', 'benefits_score']:
    if ncol in df_cleaned.columns:
        temp = pd.to_numeric(df_cleaned[ncol], errors='coerce')
        invalid_numeric = temp.isna().sum()
        print(f"{ncol}: {invalid_numeric} invalid numeric entries")


posting_date: 0 invalid date entries
application_deadline: 0 invalid date entries
years_experience: 0 invalid numeric entries
job_description_length: 0 invalid numeric entries
benefits_score: 0 invalid numeric entries


In [ ]:
if 'experience_level' in df_cleaned.columns:
    exp_map = {'EN': 'Entry-Level', 'MI': 'Mid-Level', 'SE': 'Senior-Level', 'EX': 'Executive'}
    df_cleaned['experience_level'] = df_cleaned['experience_level'].map(exp_map).fillna(df_cleaned['experience_level'])

if 'company_size' in df_cleaned.columns:
    size_map = {'S': 'Small (<50)', 'M': 'Medium (50-250)', 'L': 'Large (>250)'}
    df_cleaned['company_size'] = df_cleaned['company_size'].map(size_map).fillna(df_cleaned['company_size'])

if 'employment_type' in df_cleaned.columns:
    emp_map = {'FT': 'Full-Time', 'PT': 'Part-Time', 'CT': 'Contract', 'FL': 'Freelance'}
    df_cleaned['employment_type'] = df_cleaned['employment_type'].map(emp_map).fillna(df_cleaned['employment_type'])



In [ ]:

def remote_type_from_ratio(x):
    if pd.isna(x): return 'Unknown'
    if x == 0: return 'Onsite'
    if x == 100: return 'Fully Remote'
    if 0 < x < 100: return 'Hybrid'
    return 'Unknown'

if 'remote_ratio' in df_cleaned.columns:
    df_cleaned['remote_type'] = df_cleaned['remote_ratio'].apply(remote_type_from_ratio)

print("Standardized 'experience_level', 'company_size', 'employment_type', and created 'remote_type'.")

df.head()

Standardized 'experience_level', 'company_size', 'employment_type', and created 'remote_type'.


,job_id,job_title,salary_usd,salary_currency,experience_level,employment_type,company_location,company_size,employee_residence,remote_ratio,required_skills,education_required,years_experience,industry,posting_date,application_deadline,job_description_length,benefits_score,company_name
0,AI00001,AI Research Scientist,90376,USD,SE,CT,China,M,China,50,"Tableau, PyTorch, Kubernetes, Linux, NLP",Bachelor,9,Automotive,2024-10-18,2024-11-07,1076,5.9,Smart Analytics
1,AI00002,AI Software Engineer,61895,USD,EN,CT,Canada,M,Ireland,100,"Deep Learning, AWS, Mathematics, Python, Docker",Master,1,Media,2024-11-20,2025-01-11,1268,5.2,TechCorp Inc
2,AI00003,AI Specialist,152626,USD,MI,FL,Switzerland,L,South Korea,0,"Kubernetes, Deep Learning, Java, Hadoop, NLP",Associate,2,Education,2025-03-18,2025-04-07,1974,9.4,Autonomous Tech
3,AI00004,NLP Engineer,80215,USD,SE,FL,India,M,India,50,"Scala, SQL, Linux, Python",PhD,7,Consulting,2024-12-23,2025-02-24,1345,8.6,Future Systems
4,AI00005,AI Consultant,54624,EUR,EN,PT,France,S,Singapore,100,"MLOps, Java, Tableau, Python",Master,0,Media,2025-04-15,2025-06-23,1989,6.6,Advanced Robotics


**Feature Engineering: Skills Count**

In [ ]:
import re

def parse_skills(text):
    if pd.isna(text):
        return []
    parts = re.split(r'[;,|\n]+', str(text))
    skills = [p.strip().lower() for p in parts if p.strip()]
    return list(dict.fromkeys(skills)) 

if 'required_skills' in df_cleaned.columns:
    df_cleaned['skills_list'] = df_cleaned['required_skills'].apply(parse_skills)
    df_cleaned['skills_count'] = df_cleaned['skills_list'].apply(len)
    print("Created 'skills_list' and 'skills_count' features.")
else:
    print("Warning: 'required_skills' column not found. Skipping skill processing.")

print(df_cleaned[['required_skills', 'skills_list', 'skills_count']].head())


Created 'skills_list' and 'skills_count' features.
                                   required_skills  \
0         Tableau, PyTorch, Kubernetes, Linux, NLP   
1  Deep Learning, AWS, Mathematics, Python, Docker   
2     Kubernetes, Deep Learning, Java, Hadoop, NLP   
3                        Scala, SQL, Linux, Python   
4                     MLOps, Java, Tableau, Python   

                                         skills_list  skills_count  
0         [tableau, pytorch, kubernetes, linux, nlp]             5  
1  [deep learning, aws, mathematics, python, docker]             5  
2     [kubernetes, deep learning, java, hadoop, nlp]             5  
3                        [scala, sql, linux, python]             4  
4                     [mlops, java, tableau, python]             4  


**Numerical Imputation & Outlier Handling**

In [ ]:
df['posting_date'] = pd.to_datetime(df['posting_date'])
df['application_deadline'] = pd.to_datetime(df['application_deadline'])

In [ ]:
df['application_duration'] = (df['application_deadline'] - df['posting_date']).dt.days
df['application_duration']

0        20
1        52
2        20
3        63
4        69
         ..
14995    47
14996    14
14997    44
14998    32
14999    23
Name: application_duration, Length: 15000, dtype: int64

**Removing Outliers**

In [ ]:
# -----------------------------
# Outlier Detection Using Prediction Intervals (GBR)
# -----------------------------
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

# -----------------------------
# 1. Select numeric + categorical features
# -----------------------------
target = "salary_usd"

# Drop rows with missing target
df_model = df_cleaned.dropna(subset=[target]).copy()

# Select features (customize as needed)
feature_cols = [
    "experience_level",
    "employment_type",
    "company_location",
    "company_size",
    "employee_residence",
    "remote_ratio",
    "skills_count",
    "years_experience",
    "job_description_length",
    "benefits_score"
]

# Keep only existing columns
feature_cols = [c for c in feature_cols if c in df_model.columns]

# One-hot encode categoricals
df_encoded = pd.get_dummies(df_model[feature_cols], drop_first=True)

X = df_encoded.values
y = df_model[target].values

# -----------------------------
# 2. Train quantile models
# -----------------------------
gbr_lower = GradientBoostingRegressor(loss="quantile", alpha=0.05, n_estimators=300, max_depth=3)
gbr_middle = GradientBoostingRegressor(loss="quantile", alpha=0.50, n_estimators=300, max_depth=3)
gbr_upper = GradientBoostingRegressor(loss="quantile", alpha=0.95, n_estimators=300, max_depth=3)

gbr_lower.fit(X, y)
gbr_middle.fit(X, y)
gbr_upper.fit(X, y)

# -----------------------------
# 3. Predict intervals
# -----------------------------
lower = gbr_lower.predict(X)
pred   = gbr_middle.predict(X)
upper = gbr_upper.predict(X)

# -----------------------------
# 4. Identify outliers
# -----------------------------
df_model["lower_bound"] = lower
df_model["pred_salary"] = pred
df_model["upper_bound"] = upper

df_model["is_outlier"] = (df_model[target] < df_model["lower_bound"]) | \
                         (df_model[target] > df_model["upper_bound"])

print("Outliers detected:", df_model["is_outlier"].sum())

# -----------------------------
# 5. Merge results back into df_cleaned
# -----------------------------
df_cleaned = df_cleaned.merge(
    df_model[["job_id", "lower_bound", "pred_salary", "upper_bound", "is_outlier"]],
    on="job_id",
    how="left"
)

print("Outlier detection complete.")
df_cleaned.head()


**Save Cleaned Data**

In [ ]:
OUT_DIR = Path('data/processed')
OUT_DIR.mkdir(parents=True, exist_ok=True)
clean_path = OUT_DIR / 'new_jobs_global.csv'

df_cleaned.to_csv(clean_path, index=False)

print(f"Saved cleaned global data to: {clean_path}")
print(f"Final dataset shape: {df_cleaned.shape}")


Saved cleaned global data to: data\processed\new_jobs_global.csv
Final dataset shape: (15000, 21)
